In [1]:
# Databricks notebook source
rdd=sc.wholeTextFiles("/FileStore/tables/w1j05c2w1508921343218")
#print(rdd.take(1))

f1=rdd.map(lambda kv:(kv[0].split("/")[-1],kv[1]))

#print(f1.take(1))

f2=f1.map(lambda kv:(kv[0].split(".txt")[0],kv[1]))

#print(f2.take(1))

f3=f2.flatMapValues(lambda v: v.split("\r\n"))

#print(f3.take(1))

f4=f3.map(lambda kv:(kv[0],kv[0].split("_")[0],kv[1].split(" ")[0],kv[1].split(" ")[1]))

f5=f4.map(lambda t :(t[1],t[0],t[2],t[3]))


In [2]:
#Q1. Average monthly income of the shop in France
Q1=f3.map(lambda kv:(kv[1].split(" ")[0], float(kv[1].split(" ")[1]))) # return (month, income)

Q1_final=Q1.map(lambda x: x[1]/12) .sum()    #calculation of the sum of all income divided by 12 
print Q1_final

In [3]:
#Q2. Total revenue per city per year
Q2=f3.map(lambda kv:(kv[0].split("_")[0], float(kv[1].split(" ")[1]))) #key : city, val = income
Q2_final=Q2.reduceByKey(lambda val1,val2 : val1+val2)
print Q2_final.collect()

#Other Method, same result
#Q2_final=f6.groupByKey().mapValues(lambda x: sum(x) ).collect()
#print Q2_final

In [4]:
#Q3. Average monthly income of the shop in each city

Q3=f4.map(lambda kv:(kv[1], float(kv[3]))) # return key = name of the shop without the number
                                           #        val = income
Q3_final=Q3.groupByKey().mapValues(lambda x: sum(x) / len(x)).collect()    #calculation of the average by key

print Q3_final

In [5]:
#Q4. Total revenue per store per year
Q4=f4.map(lambda x: (x[0], float (x[3]) )) #return key = shop, value = income

Q4_final=Q4.groupByKey().mapValues(lambda x: sum(x) ).collect()    #calculation of the sum by key
print Q4_final

In [7]:
 #Q5.The store that achieves the best performance in each month
Q5=f4.map(lambda x: (x[2], (x[0] ,x[3]))) #return key= month, value =(shop, income)
Q5_final=test.reduceByKey(lambda value1, value2:( value1 if value1[1] > value2[1] else value2))\
.map(lambda paire:(paire[0],paire[1][0],paire[1][1]))
print Q5_final.collect()